In [1]:
import os 
BASE_DIR=os.path.abspath("../../")
import sys 
sys.path.append(BASE_DIR)
DATA_DIR="/home/arturo/Documents/programacion_stuff/DeepElasticity/data/002-Neo"
from src.utils.data import DataNeo_2
from src.models import PINN  
from src.train import Trainer_2
import torch 
import matplotlib.pyplot as plt 
import time
from datetime import datetime
import pickle
import numpy as np  

In [2]:
data=DataNeo_2("DATOS_HIPERELASTICO_3",3,E="E10kPa",base_dir=DATA_DIR)
data.load_data(load_stage=3)


In [3]:
data.final_data_pd

,X,Y,Z,S1,S2,S3,S12,S23,S13,Direccion_Fuerza,Fuerza,UX,UY,UZ,Restricciones,Final_X,Final_Y,Final_Z
Node,,,,,,,,,,,,,,,,,,
1.0,46.726,41.840,17.459,6.055700e-05,-0.000457,-0.000023,0.000163,0.000075,0.000011,NaN,NaN,-0.010518,-0.311773,0.017699,NaN,46.736518,42.151773,17.441301
2.0,31.149,44.342,17.410,-1.159959e-04,-0.001008,0.000002,-0.000313,0.000123,0.000057,NaN,NaN,0.052825,-0.358916,0.026674,NaN,31.096175,44.700916,17.383326
3.0,54.619,26.576,39.528,-3.743202e-05,-0.000704,-0.000024,0.000063,-0.000006,-0.000106,NaN,NaN,0.011489,0.000000,-0.018204,[UY],54.607511,26.576000,39.546204
4.0,63.720,39.982,42.026,1.468523e-08,0.000069,0.000049,0.000011,-0.000212,0.000029,NaN,NaN,0.010692,-0.039530,-0.006892,NaN,63.709308,40.021530,42.032892
5.0,19.543,50.820,41.746,1.437263e-04,-0.000087,0.000628,0.000160,-0.000578,0.000639,NaN,NaN,0.079790,-0.065497,0.055505,NaN,19.463210,50.885497,41.690495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4147.0,47.957,36.969,40.653,2.301579e-04,-0.001318,0.000157,0.000450,-0.000128,-0.000038,NaN,NaN,0.022689,-0.180199,-0.001721,NaN,47.934311,37.149199,40.654721
4148.0,43.244,36.666,35.819,3.805502e-04,-0.001610,0.000109,0.000308,-0.000121,-0.000027,NaN,NaN,0.014009,-0.211796,0.003708,NaN,43.229991,36.877796,35.815292
4149.0,46.140,37.853,37.430,2.816971e-04,-0.001450,0.000137,0.000521,-0.000183,-0.000031,NaN,NaN,0.020318,-0.211139,-0.000234,NaN,46.119682,38.064139,37.430234


In [4]:
train_init_pos_main,train_disp_main,test_init_pos_main,test_disp_main,position_selected_stresses,return_stress,return_colloc_points=data.prepare_pytorch_data()


In [5]:
from src.models import PINN_NeoHook

pinn=PINN_NeoHook([3,25,25,25,25,3],0.4,0.01)
step_dict = {
    "step_1": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-2), 
            "epochs": 15},
    "step_2": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-3), 
            "epochs": 15},
    "step_3": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-4), 
            "epochs": 15}
}

In [6]:
trainer=Trainer_2(step_dict)
trainer.train(pinn,data)

step_1


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.81 GiB total capacity; 2.50 GiB already allocated; 18.88 MiB free; 3.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF